In [3]:
def alphabet_map(string_or_array):

    if isinstance(string_or_array, str):
        num_array = []
        list(string_or_array)
        for i in string_or_array.lower():
            num_array.append(ord(i)-97)
        return(num_array)
    else:
        word_array = []
        for i in string_or_array:
            word_array.append(chr(i+97))
        ret_string = ""
        return(ret_string.join(word_array))

print(alphabet_map("test"), alphabet_map([19, 4, 18, 19]))

[19, 4, 18, 19] test


In [4]:
def affine(cipher_string, alpha, beta, decode):
    valid = {True, False}

    if decode not in valid:
        raise ValueError("results: status must be one of %r." % valid)
    
    i_array = alphabet_map(cipher_string)
    o_array = []

    if decode:
        for i in i_array:
            gamma = pow(alpha, -1, 26)
            o_array.append(gamma * (i - beta) % 26)
        return(alphabet_map(o_array))
    else:
        cipher_string = cipher_string.lower()
        for i in i_array:
            o_array.append((alpha*i + beta ) %26)
        return(alphabet_map(o_array).upper())

print(affine('YXCYIS', 19, 12, True), affine('crucio', alpha= 19, beta = 12, decode = False))

crucio YXCYIS


In [5]:
import re

def find_key_size(cipher_text):
    regex = re.compile('[^a-zA-Z]')
    cipher_text = regex.sub('', cipher_text)
    cipher_arr = list(cipher_text)

    shift_dict = {}
    for shift_value in range(1,len(cipher_arr)):
        match = 0
        for i in range(len(cipher_arr)):
            if cipher_arr[i] == cipher_arr[(i + shift_value) % len(cipher_arr)]:
                match += 1
        shift_dict.update({shift_value:match})
    try:
        return max(shift_dict, key=shift_dict.get)
    except:
        return 0

print(f'the most likely key size is: {str(find_key_size("CBCABAAACA"))} ')

the most likely key size is: 2 


In [6]:
import numpy as np

# helper function from #9 originally but it's useful here since we're doing the same thing
def frequency(txt, sign):
    counter: int = 0
    for s in txt:
        if s != sign:
            continue
        counter += 1
    return counter


def find_key(cipher_text):
    key_length = find_key_size(cipher_text)
    cipher_arr = np.array(list(cipher_text))

    string_array = []
    dict_array = []
    key_array = []

    for mod_val in range(key_length):
        string_array.append(cipher_arr[mod_val::key_length])

    for string in string_array:
        value_dict = {}

        for s in 'ABCDEFGHIJKLMNOPQRSTUVWXYZ':
            word_freq = frequency(string, s)
            percent = 100 * word_freq / len(string)
            value_dict.update({s: percent})

        value_dict = {k: v for k, v in sorted(
            value_dict.items(), key=lambda item: item[1], reverse=True)}

        dict_array.append(value_dict)

    for word_dict in dict_array:
        val = list(word_dict.keys())[0]
        key_array.append(val)

    str = ""
    return str.join(key_array)

#key is CA
print("A likely key is: " + find_key("CBCABAAACA"))

A likely key is: CA


In [7]:
def EEA(a, b):
    if a == 0:
        return b, 0, 1
    else:
        gcd, x, y = EEA(b % a, a)
        return gcd, y - (b // a) * x, x
 
a = 2
b = 5

gcd, x, y = EEA(a, b)
print('The GCD is', gcd)
print(f'{gcd} = {x}({a}) + {y}({b})')

The GCD is 1
1 = -2(2) + 1(5)


In [8]:
import numpy as np
from math import ceil, log2, floor

def mod_exp(a,b,n): # for a^b mod(n)
    pow_dict = [pow(2,i) for i in range(floor(log2(b)),-1,-1)]
    binary_b = [int(x) for x in list(format(b, "b"))]
    return np.prod([pow(a,x*y) % n for x,y in zip(pow_dict, binary_b)]) % n

mod_exp(43855823121249, 22, 23)

1

In [9]:
def prime_factors(n):
    i = 2
    factors = []
    while i * i <= n:
        if n % i:
            i += 1
        else:
            n //= i
            factors.append(i)
    if n > 1:
        factors.append(n)
    return factors

prime_factors(100)

[2, 2, 5, 5]